In [20]:
import pandas as pd
import datetime as dt
import numpy as np

In [21]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202540.csv")

In [22]:
ema = ema.copy()

# -------------------------------
# 1. Calcular venta_costo_ema
# -------------------------------
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# -------------------------------
# 2. Totales por sucursal
# -------------------------------
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# -------------------------------
# 3. Proporciones dentro de cada sucursal
# -------------------------------
ema["venta_costo%"] = (ema["venta_costo_ema"] / ema["venta_costo_tot"]).round(4)
ema["cantidad%"] = (ema["EMA"] / ema["total_ema"]).round(4)

# -------------------------------
# 4. Orden y acumulados
# -------------------------------
# Ordenar por costo dentro de cada sucursal
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviación por sucursal
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = (
        ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)
    )

# -------------------------------
# 5. Renombrar columnas (estilo reporte Odoo)
# -------------------------------
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# -------------------------------
# 6. Calcular desviación en dinero
# -------------------------------
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (
        ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]
    ).round(1)



# Ejemplo de reglas (lo que viene de conf_acumulado_desviacion en Odoo) #PERILLA
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

# aplicar reglas 0–0.8–1 como antes, pero sobre coef_ventas_norm


# Función que asigna el valor según reglas
def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return np.nan  # si no entra en ningún rango

# Aplicar
ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].apply(
    lambda x: asignar_desviacion(x, reglas)
)



# -------------------------------
# 7. Limpiar columnas innecesarias
# -------------------------------
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])


# 8. Reglas acumulado costo combinado
# --- Configuración ---
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

## PERILLA

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

# Valor de referencia en porcentaje (ejemplo: 10%) ##PERILLA
conf_acumulado_cantidad = 80
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)


# --- Cálculo en el DataFrame ---
def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

# Nueva columna en tu df
ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(2)

ema_def["acumulado_combinado"] = (
    ema_def.groupby("store_name")["acumulado_combinado"]
    .transform(lambda x: x / x.max())
).round(2)


def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)


In [23]:
ema_ord = ema_def[["store_name","product_ref","Año","semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

ema_ord = ema_ord[~(ema_ord["store_name"]=="0")]

In [24]:
ema_ord["acumulado_combinado"].describe()

count    431360.000000
mean          0.978722
std           0.087132
min           0.010000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: acumulado_combinado, dtype: float64

In [25]:
ema_ord[(ema_ord["store_name"]=="SUCURSAL NORTE") & (ema_ord["product_ref"]=="DAE02286025")]

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
283885,SUCURSAL NORTE,DAE02286025,2025.0,38,1.639138,97279.53,159454.57,0.0007,0.2231,1.388627e+10,87086.03,0.51,0.0,5968365.96,0.0,0.36,AAA
283886,SUCURSAL NORTE,DAE02286025,2025.0,39,1.311310,97279.53,127563.62,0.0006,0.2603,1.122560e+05,0.88,0.56,12.0,7079449.66,1.0,0.62,A
283887,SUCURSAL NORTE,DAE02286025,2025.0,40,1.049048,97279.53,102050.90,0.0005,0.3125,1.122560e+05,1.10,0.62,13.0,10305581.40,1.0,0.67,A
283888,SUCURSAL NORTE,DAE02286025,2025.0,41,0.839238,97279.53,81640.68,0.0004,0.3691,9.960160e+04,1.22,0.67,14.0,13289533.57,1.0,0.72,A
283889,SUCURSAL NORTE,DAE02286025,2025.0,42,0.671390,97279.53,65312.50,0.0003,0.4271,8.425310e+04,1.29,0.72,15.0,15264258.08,1.0,0.77,A
283874,SUCURSAL NORTE,DAE02286025,2025.0,27,0.320000,97279.53,31129.45,0.0001,0.6424,0.000000e+00,0.00,0.90,0.0,27218747.35,0.0,0.91,B
283875,SUCURSAL NORTE,DAE02286025,2025.0,28,0.256000,97279.53,24903.56,0.0001,0.6926,0.000000e+00,0.00,0.90,0.0,31729735.48,0.0,0.93,B
283876,SUCURSAL NORTE,DAE02286025,2025.0,29,0.204800,97279.53,19922.85,0.0001,0.7461,9.016052e+08,45254.83,0.90,0.0,38391670.31,0.0,0.96,C
283877,SUCURSAL NORTE,DAE02286025,2025.0,30,0.184320,97279.53,17930.56,0.0001,0.7720,1.034923e+09,57718.40,0.90,0.0,40922241.33,0.0,0.96,C
283878,SUCURSAL NORTE,DAE02286025,2025.0,31,0.165888,97279.53,16137.51,0.0001,0.7992,9.766749e+08,60522.03,0.90,0.0,42342455.15,0.0,0.97,C


## NIVEL DE SERVICIO

In [26]:
# Definimos las condiciones para la meta, basadas en la clasificación
condiciones_meta = [
    (ema_ord['Clasificacion'] == 'AAA'),
    (ema_ord['Clasificacion'] == 'A'),
    (ema_ord['Clasificacion'] == 'B'),
    (ema_ord['Clasificacion'] == 'C')
]

# Definimos los valores de la meta que se asignarán a cada clasificación
metas = [0.999, 0.98, 0.90, 0.50]

ema_ord = ema_ord.copy()

# Asignamos la nueva columna 'Meta' usando np.select
ema_ord.loc[:, 'Nivel de servicio'] = np.select(condiciones_meta, metas, default=np.nan)

In [27]:
ema_ord.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv")

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

bq.head((5))

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
205258,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,32,60.760150,13771.02,836729.24,0.0009,0.1002,150611.3,0.18,0.00,1.818182,0.18,0.8,0.22,AAA,0.999
205267,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,41,54.888271,13771.02,755867.48,0.0008,0.1096,173849.5,0.23,0.00,4.512195,0.41,1.0,0.26,AAA,0.999
205259,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,33,54.808120,13771.02,754763.72,0.0008,0.1104,188690.9,0.25,0.01,3.867596,0.66,1.0,0.28,AAA,0.999
205260,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,34,52.646496,13771.02,724995.95,0.0008,0.1136,202998.9,0.28,0.01,4.072327,0.94,1.0,0.28,AAA,0.999
246767,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,41,51.032786,9917.17,506100.81,0.0006,0.1945,146769.2,0.29,0.01,4.633803,1.23,1.0,0.30,AAA,0.999


In [28]:
df_mio = ema_ord[["store_name","product_ref","semana","Ema","acumulado_combinado","Clasificacion"]]


In [29]:
conteo = pd.crosstab(df_mio["store_name"], df_mio["Clasificacion"])
conteo

Clasificacion,A,AAA,B,C
store_name,,,,
PRINCIPAL COTA,1520,375,1779,50246
SUCURSAL BARRANQUILLA,1854,593,2178,49295
SUCURSAL BUCARAMANGA,2009,657,2251,49003
SUCURSAL CALI,1782,495,2138,49505
SUCURSAL CALLE 6,1772,709,2121,49318
SUCURSAL MEDELLIN,1506,481,1978,49955
SUCURSAL NORTE,1617,471,2031,49801
SUCURSAL VALLADOLID,1438,417,1917,50148


In [30]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","acumulado_combinado","clasificacion","almacen_id" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)

df_real =df.copy()


In [31]:
df_mio.head()

,store_name,product_ref,semana,Ema,acumulado_combinado,Clasificacion
400256,PRINCIPAL COTA,DCS10536137,31,27.875727,0.23,AAA
71650,PRINCIPAL COTA,BCE00606125,32,26.149581,0.03,AAA
400257,PRINCIPAL COTA,DCS10536137,32,25.088154,0.20,AAA
400253,PRINCIPAL COTA,DCS10536137,28,24.298666,0.02,AAA
400267,PRINCIPAL COTA,DCS10536137,42,22.763445,0.24,AAA


In [32]:
df_real = df_real[["store_name","product_name","semana","ema","acumulado_combinado","clasificacion"]]

In [33]:
df_real

,store_name,product_name,semana,ema,acumulado_combinado,clasificacion
0,SUCURSAL BARRANQUILLA,DAB02570025,41,55.576436,0.255344,AAA
1,SUCURSAL BARRANQUILLA,DAB14570025,41,49.148446,0.258879,AAA
2,SUCURSAL BARRANQUILLA,BCS00035125,41,30.721506,0.245765,AAA
3,SUCURSAL BARRANQUILLA,BCS00025125,41,29.862435,0.231440,AAA
4,SUCURSAL BARRANQUILLA,DAB02772025,41,29.148903,0.353142,AAA
...,...,...,...,...,...,...
43852,SUCURSAL NORTE,DAE05466003,41,0.000000,1.000000,C
43853,SUCURSAL NORTE,DAE05094003,41,0.000000,1.000000,C
43854,SUCURSAL NORTE,DAE06511003,41,0.000000,1.000000,C
43855,SUCURSAL NORTE,DAE06627002,41,0.000000,1.000000,C
